In [ ]:
with open("sample_emails.txt",encoding = 'utf-8') as f:
  sample_corpus = f.read()

In [ ]:
sample_corpus

'From r  Wed Oct 30 21:41:56 2002\nReturn-Path: <james_ngola2002@maktoob.com>\nX-Sieve: cmu-sieve 2.0\nReturn-Path: <james_ngola2002@maktoob.com>\nMessage-Id: <200210310241.g9V2fNm6028281@cs.CU>\nFrom: "MR. JAMES NGOLA." <james_ngola2002@maktoob.com>\nReply-To: james_ngola2002@maktoob.com\nTo: webmaster@aclweb.org\nDate: Thu, 31 Oct 2002 02:38:20 +0000\nSubject: URGENT BUSINESS ASSISTANCE AND PARTNERSHIP\nX-Mailer: Microsoft Outlook Express 5.00.2919.6900 DM\nMIME-Version: 1.0\nContent-Type: text/plain; charset="us-ascii"\nContent-Transfer-Encoding: 8bit\nX-MIME-Autoconverted: from quoted-printable to 8bit by sideshowmel.si.UM id g9V2foW24311\nStatus: O\n\nFROM:MR. JAMES NGOLA.\nCONFIDENTIAL TEL: 233-27-587908.\nE-MAIL: (james_ngola2002@maktoob.com).\n\nURGENT BUSINESS ASSISTANCE AND PARTNERSHIP.\n\n\nDEAR FRIEND,\n\nI AM ( DR.) JAMES NGOLA, THE PERSONAL ASSISTANCE TO THE LATE CONGOLESE (PRESIDENT LAURENT KABILA) WHO WAS ASSASSINATED BY HIS BODY GUARD ON 16TH JAN. 2001.\n\n\nTHE INCIDE

### Extract Email Information

In [ ]:
import re
match = re.findall(r'From:.*',sample_corpus )

In [ ]:
match

['From: "MR. JAMES NGOLA." <james_ngola2002@maktoob.com>',
 'From: "Mr. Ben Suleman" <bensul2004nng@spinfinder.com>',
 'From: "PRINCE OBONG ELEME" <obong_715@epatra.com>']

### Extract Name

In [ ]:
match = re.findall(r'From:\s*\"(.*)\"',sample_corpus)
print(*match, sep='\n')

MR. JAMES NGOLA.
Mr. Ben Suleman
PRINCE OBONG ELEME


### Extract email

In [ ]:
match = re.findall(r'From: .* \<(.*)\>',sample_corpus)
print(*match, sep='\n')

james_ngola2002@maktoob.com
bensul2004nng@spinfinder.com
obong_715@epatra.com


### Extract mail

In [ ]:
match = re.findall(r'From:.*<\w+@(\w+).com>',sample_corpus)
print(*match, sep='\n')

m, a, k, t, o, o, b
s, p, i, n, f, i, n, d, e, r
e, p, a, t, r, a


In [ ]:
match = re.findall(r'From:.*<(\w+)@(\w+).com>',sample_corpus)
print(*[', '.join(m) for m in match], sep='\n')

james_ngola2002, maktoob
bensul2004nng, spinfinder
obong_715, epatra


## Use Kaggle Fraud Emails 

In [30]:
import re
import pandas as pd
import email

with open('all_emails.txt','r', encoding='windows-1252') as f:
    text = f.read()
emails = re.split(r'From r',text)[1:]
len(emails)

3977

In [42]:
emails[0]

'  Wed Oct 30 21:41:56 2002\nReturn-Path: <james_ngola2002@maktoob.com>\nX-Sieve: cmu-sieve 2.0\nReturn-Path: <james_ngola2002@maktoob.com>\nMessage-Id: <200210310241.g9V2fNm6028281@cs.CU>\nFrom: "MR. JAMES NGOLA." <james_ngola2002@maktoob.com>\nReply-To: james_ngola2002@maktoob.com\nTo: webmaster@aclweb.org\nDate: Thu, 31 Oct 2002 02:38:20 +0000\nSubject: URGENT BUSINESS ASSISTANCE AND PARTNERSHIP\nX-Mailer: Microsoft Outlook Express 5.00.2919.6900 DM\nMIME-Version: 1.0\nContent-Type: text/plain; charset="us-ascii"\nContent-Transfer-Encoding: 8bit\nX-MIME-Autoconverted: from quoted-printable to 8bit by sideshowmel.si.UM id g9V2foW24311\nStatus: O\n\nFROM:MR. JAMES NGOLA.\nCONFIDENTIAL TEL: 233-27-587908.\nE-MAIL: (james_ngola2002@maktoob.com).\n\nURGENT BUSINESS ASSISTANCE AND PARTNERSHIP.\n\n\nDEAR FRIEND,\n\nI AM ( DR.) JAMES NGOLA, THE PERSONAL ASSISTANCE TO THE LATE CONGOLESE (PRESIDENT LAURENT KABILA) WHO WAS ASSASSINATED BY HIS BODY GUARD ON 16TH JAN. 2001.\n\n\nTHE INCIDENT OCC

In [45]:
emails_list = []

for mail in emails[:20]:
  # Extract Sender name and address
  emails_dict = dict() 
  # Sender
  sender = re.search(r'From:.*',mail)
  # Check if errors
  if sender:
    sender_mail = re.search(r'\w\S+@.*\b',sender.group())
    sender_name = re.search(r'(?<=\").*(?=\")',sender.group())
  else:
    sender_mail, sender_name = None, None
  # put in dict
  if (sender_mail and sender_name):
    emails_dict['sender_email'] = sender_mail.group()
    emails_dict['sender_name'] = sender_name.group()
  
  # Extract Reciever name and address
  receiver = re.search(r'To:.*', mail)
  if receiver:
    receiver_mail = re.search(r'\w\S+@.*\b',receiver.group())
    receiver_name = re.search(r'(?<=\").*(?=\")',receiver.group())
  else:
    receiver_mail, receiver_name = None, None

  if (receiver_mail and receiver_name):
    emails_dict['receiver_email'] = receiver_mail.group()
    emails_dict['reciever_name'] = receiver_name.group()

  # Extract Date
  date_match = re.search(r'Date:.*', mail)
  if date_match:
    date = re.search(r'\d+\s\w+\s\d+', date_match.group())
  else:
    date = None

  if date:
    emails_dict['sent_date'] = date.group()

   #Extract Subject 
  subject_match = re.search(r'Subject:.*',mail)
  subject = re.sub(r'Subject: ','',subject_match.group())

  if subject_match:
    emails_dict["subject"] = subject
  else:
    emails_dict["subject"] = None
  #use email_package to get email
  try:
    full_email = email.message_from_string(mail)
    body = full_email.get_payload()
    emails_dict["email_body"] = body
  except:
    emails_dict["email_body"] = None

  emails_list.append(emails_dict)


In [46]:
emails_df = pd.DataFrame(emails_list)
emails_df

,sender_email,sender_name,sent_date,subject,email_body
0,james_ngola2002@maktoob.com,MR. JAMES NGOLA.,31 Oct 2002,URGENT BUSINESS ASSISTANCE AND PARTNERSHIP,FROM:MR. JAMES NGOLA.\nCONFIDENTIAL TEL: 233-2...
1,bensul2004nng@spinfinder.com,Mr. Ben Suleman,31 Oct 2002,URGENT ASSISTANCE /RELATIONSHIP (P),"Dear Friend,\n\nI am Mr. Ben Suleman a custom ..."
2,obong_715@epatra.com,PRINCE OBONG ELEME,31 Oct 2002,GOOD DAY TO YOU,FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF EL...
3,obong_715@epatra.com,PRINCE OBONG ELEME,31 Oct 2002,GOOD DAY TO YOU,FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF EL...
4,m_abacha03@www.com,Maryam Abacha,1 Nov 2002,I Need Your Assistance.,"Dear sir, \n \nIt is with a heart full of hope..."
5,NaN,NaN,02 Nov 2002,Partnership,ATTENTION: ...
6,tunde_dosumu@lycos.com,Barrister tunde dosumu,NaN,Urgent Attention,"Dear Sir,\n\nI am Barrister Tunde Dosumu (SAN)..."
7,william2244drallo@maktoob.com,William Drallo,3 Nov 2002,URGENT BUSINESS PRPOSAL,FROM: WILLIAM DRALLO.\nCONFIDENTIAL TEL: 233-2...
8,abdul_817@rediffmail.com,MR USMAN ABDUL,04 Nov 2002,THANK YOU,"CHALLENGE SECURITIES LTD.\nLAGOS, NIGERIA\n\n\..."
9,barrister_td@lycos.com,Tunde Dosumu,NaN,Urgent Assistance,"Dear Sir,\n\nI am Barrister Tunde Dosumu (SAN)..."
